In [ ]:
pip install -q "weaviate-client[embedded]"

## Setup: Imports and Environment

In [6]:
import os
import sys
import time
from pathlib import Path

# Add project root to path (same pattern as other notebooks)
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))
os.chdir(project_root)

print(f"✓ Project root: {project_root}")

✓ Project root: /content/operation-ledger-mind


## Load OpenAI API key (Colab secrets or .env)

In [7]:
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✓ OpenAI API key loaded from Colab secrets")
except Exception:
    from dotenv import load_dotenv
    load_dotenv(project_root / '.env')
    print("✓ Environment loaded from .env")

print(f"✓ OPENAI_API_KEY set: {'Yes' if os.environ.get('OPENAI_API_KEY') else 'No'}")

✓ OpenAI API key loaded from Colab secrets
✓ OPENAI_API_KEY set: Yes


## Import project modules

In [8]:
from src.utils.config_loader import load_config
from src.ingestion.pdf_loader import load_pdf, clean_text
from src.ingestion.chunker import chunk_text

# RAG modules
from src.rag.weaviate_store import connect_weaviate, ensure_collection
from src.rag.index_builder import ensure_index_built, build_index
from src.rag.retrieval import dense_search, bm25_search
from src.rag.fusion import rrf_fusion
from src.rag.reranker import rerank
from src.rag.generation import generate_answer, build_rag_prompt
from src.rag.librarian_inference import query_librarian

from sentence_transformers import SentenceTransformer, CrossEncoder

print("✓ All imports successful")

✓ All imports successful


## Step 1: Load Configuration

In [9]:
config_path = project_root / 'config' / 'config.yaml'
config = load_config(config_path)

rag_cfg = config['rag']

# Resolve PDF path
raw_data = project_root / config['environment']['paths']['raw_data']
doc_name = config['project']['document']
pdf_path = raw_data / doc_name

print("✓ Configuration loaded")
print(f"  PDF: {pdf_path} (exists: {pdf_path.exists()})")
print(f"  Vector DB: {rag_cfg['vector_db']['provider']} ({rag_cfg['vector_db']['mode']})")
print(f"  Embedding model: {rag_cfg['embeddings']['model']}")
print(f"  Reranker: {rag_cfg['refinement']['reranker']['model']}")
print(f"  RRF k: {rag_cfg['refinement']['rrf']['k']}")
print(f"  Retrieval top-k: {rag_cfg['retrieval']['top_k']}")
print(f"  Reranker top-k: {rag_cfg['refinement']['reranker']['top_k']}")
print(f"  Generator mode: {rag_cfg['inference'].get('generator_mode', 'openai')}")
print(f"  Answer LLM: {rag_cfg['inference']['answer_llm']['model']}")

✓ Configuration loaded
  PDF: /content/operation-ledger-mind/data/raw/2024-Annual-Report.pdf (exists: True)
  Vector DB: weaviate (embedded)
  Embedding model: sentence-transformers/all-MiniLM-L6-v2
  Reranker: cross-encoder/ms-marco-MiniLM-L-6-v2
  RRF k: 60
  Retrieval top-k: 20
  Reranker top-k: 5
  Generator mode: openai
  Answer LLM: gpt-4o-mini


## Step 2: Build / Load Weaviate Index

This loads the PDF, chunks it, embeds with SentenceTransformers, and upserts
into Weaviate. Idempotent: if the index already exists, it skips rebuilding.

In [10]:
# Load the embedding model (cached for reuse)
embedder = SentenceTransformer(rag_cfg['embeddings']['model'])
print(f"✓ Embedder loaded: {rag_cfg['embeddings']['model']}")
print(f"  Embedding dimension: {embedder.get_sentence_embedding_dimension()}")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✓ Embedder loaded: sentence-transformers/all-MiniLM-L6-v2
  Embedding dimension: 384


In [12]:
# Connect to Weaviate and build/verify the index
client = connect_weaviate(config)
print(f"✓ Weaviate client connected ({rag_cfg['vector_db']['mode']} mode)")

count = ensure_index_built(
    config,
    client=client,
    embedder=embedder,
    force_rebuild=False,  # Set True to rebuild from scratch
    verbose=True,
)
print(f"\n✓ Weaviate index ready: {count} chunks indexed")

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 4985


✓ Weaviate client connected (embedded mode)
📄 Loading PDF: data/raw/2024-Annual-Report.pdf
✂️  Cleaned text: 621,413 chars
🧩 Chunks: 476


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

📐 Embeddings: 476 vectors of dim 384
⬆️  Upserting 476 objects into Weaviate …


Indexing: 100%|██████████| 476/476 [00:00<00:00, 2583.81it/s]


✅ Weaviate collection 'FinancialDocument' now has 476 objects

✓ Weaviate index ready: 476 chunks indexed


## Step 3: Dense Retrieval Demo

Dense (vector) retrieval encodes the question and finds chunks with the
closest embedding vectors. Good for semantic/paraphrase queries but can
miss exact entity names.

In [13]:
test_query = "What was the total revenue reported in the annual report?"

dense_results = dense_search(
    test_query,
    client=client,
    embedder=embedder,
    config=config,
    top_n=10,
)

print(f"🔵 Dense search: '{test_query}'")
print(f"   Retrieved {len(dense_results)} results\n")
for i, r in enumerate(dense_results[:5], 1):
    print(f"  [{i}] chunk {r['meta']['chunk_id']} "
          f"(score: {r['score']:.4f}) "
          f"pp. {r['meta']['page_start']}-{r['meta']['page_end']}")
    print(f"      {r['content'][:150]}...\n")

🔵 Dense search: 'What was the total revenue reported in the annual report?'
   Retrieved 10 results

  [1] chunk 236 (score: 0.5168) pp. 0-0
      1,273
19 %
Gross Bookings (2)
$ 137,865 $ 162,773
18 %
21 %
Revenue
$
37,281 $
43,978
18 %
19 %
Income from operations
$
1,110 $
2,799
152 %
Net incom...

  [2] chunk 259 (score: 0.5117) pp. 0-0
      2023 to 2024 %
Change
(In millions, except percentages)

2023

2024

Mobility
$
19,832 $
25,087
26 %
Delivery

12,204
13,750
13 %
Freight

5,245
5,141...

  [3] chunk 280 (score: 0.4950) pp. 0-0
      iabilities,
revenue and expenses, and related disclosures in our audited consolidated financial statements. We have based our estimates on
histor
cal ...

  [4] chunk 432 (score: 0.4923) pp. 0-0
      gistics services
offerings.
For information about how our reportable segments derive revenue, as well as revenue grouped by offerings and geographical...

  [5] chunk 258 (score: 0.4917) pp. 0-0
      exceed our ability to use these credits in future

## Step 4: BM25 Retrieval Demo

BM25 (keyword/sparse) retrieval matches exact terms. Essential for
entity-heavy financial queries like "Form 10-K" or "$37B".

In [14]:
bm25_results = bm25_search(
    test_query,
    client=client,
    config=config,
    top_n=10,
)

print(f"🟠 BM25 search: '{test_query}'")
print(f"   Retrieved {len(bm25_results)} results\n")
for i, r in enumerate(bm25_results[:5], 1):
    print(f"  [{i}] chunk {r['meta']['chunk_id']} "
          f"(score: {r['score']:.4f}) "
          f"pp. {r['meta']['page_start']}-{r['meta']['page_end']}")
    print(f"      {r['content'][:150]}...\n")

🟠 BM25 search: 'What was the total revenue reported in the annual report?'
   Retrieved 10 results

  [1] chunk 0 (score: 4.7073) pp. 0-0
      On Our Way
Uber’s Mission
We reimagine the way the world moves for the better
We are Uber.
The go-getters.
The kind of people who are relentless about...

  [2] chunk 280 (score: 4.2063) pp. 0-0
      iabilities,
revenue and expenses, and related disclosures in our audited consolidated financial statements. We have based our estimates on
histor
cal ...

  [3] chunk 258 (score: 3.8948) pp. 0-0
      exceed our ability to use these credits in future periods.
Provision for income taxes decreased by $6.0 billion primarily attributable to
the release ...

  [4] chunk 259 (score: 3.7643) pp. 0-0
      2023 to 2024 %
Change
(In millions, except percentages)

2023

2024

Mobility
$
19,832 $
25,087
26 %
Delivery

12,204
13,750
13 %
Freight

5,245
5,141...

  [5] chunk 249 (score: 3.7610) pp. 0-0
      ily includes the results of our share of income or l

In [15]:
# Compare: which chunks appear in dense but not BM25 and vice versa?
dense_ids = {r['id'] for r in dense_results}
bm25_ids = {r['id'] for r in bm25_results}

print(f"Dense-only chunks: {len(dense_ids - bm25_ids)}")
print(f"BM25-only chunks:  {len(bm25_ids - dense_ids)}")
print(f"Overlap:           {len(dense_ids & bm25_ids)}")
print(f"\n→ This overlap gap is why hybrid search matters!")

Dense-only chunks: 6
BM25-only chunks:  6
Overlap:           4

→ This overlap gap is why hybrid search matters!


## Step 5: Reciprocal Rank Fusion (RRF)

RRF combines the two ranked lists into one. Formula:

$$\text{RRF}(d) = \sum_{i} \frac{1}{k + \text{rank}_i(d)}$$

Docs that rank high in **both** lists get the highest fused scores.

In [16]:
rrf_k = int(config['rag']['refinement']['rrf']['k'])

fused_results = rrf_fusion(dense_results, bm25_results, k=rrf_k)

print(f"🔀 RRF Fusion (k={rrf_k}): {len(fused_results)} unique candidates\n")
for i, r in enumerate(fused_results[:8], 1):
    print(f"  [{i}] chunk {r['meta']['chunk_id']} "
          f"RRF={r['rrf_score']:.4f} "
          f"(dense_rank={r.get('dense_rank', '-')}, bm25_rank={r.get('bm25_rank', '-')})")
    print(f"      {r['content'][:120]}...\n")

🔀 RRF Fusion (k=60): 16 unique candidates

  [1] chunk 280 RRF=0.0320 (dense_rank=3, bm25_rank=2)
      iabilities,
revenue and expenses, and related disclosures in our audited consolidated financial statements. We have base...

  [2] chunk 259 RRF=0.0318 (dense_rank=2, bm25_rank=4)
      2023 to 2024 %
Change
(In millions, except percentages)

2023

2024

Mobility
$
19,832 $
25,087
26 %
Delivery

12,204
13...

  [3] chunk 258 RRF=0.0313 (dense_rank=5, bm25_rank=3)
      exceed our ability to use these credits in future periods.
Provision for income taxes decreased by $6.0 billion primaril...

  [4] chunk 249 RRF=0.0301 (dense_rank=8, bm25_rank=5)
      ily includes the results of our share of income or loss from our
quity
method investments. For additional information, s...

  [5] chunk 236 RRF=0.0164 (dense_rank=1, bm25_rank=None)
      1,273
19 %
Gross Bookings (2)
$ 137,865 $ 162,773
18 %
21 %
Revenue
$
37,281 $
43,978
18 %
19 %
Income from operations
$...

  [6] chunk 0 RRF=0.0164

## Step 6: Cross-Encoder Reranking

The cross-encoder sees (query, document) **together** – unlike the bi-encoder
which embeds them separately. This gives higher accuracy but is too slow
for the full corpus, so we only apply it to the top fused candidates.

In [17]:
# Load cross-encoder reranker
reranker_model = CrossEncoder(rag_cfg['refinement']['reranker']['model'])
rerank_top_k = int(rag_cfg['refinement']['reranker']['top_k'])

print(f"✓ Reranker loaded: {rag_cfg['refinement']['reranker']['model']}")

reranked_results = rerank(
    test_query,
    fused_results[:20],  # rerank top-20 fused candidates
    cross_encoder=reranker_model,
    top_k=rerank_top_k,
)

print(f"\n🏆 Reranked to top-{len(reranked_results)}:\n")
for i, r in enumerate(reranked_results, 1):
    print(f"  [{i}] chunk {r['meta']['chunk_id']} "
          f"rerank_score={r['rerank_score']:.3f} "
          f"(RRF={r.get('rrf_score', 0):.4f})")
    print(f"      {r['content'][:150]}...\n")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✓ Reranker loaded: cross-encoder/ms-marco-MiniLM-L-6-v2

🏆 Reranked to top-5:

  [1] chunk 259 rerank_score=-1.130 (RRF=0.0318)
      2023 to 2024 %
Change
(In millions, except percentages)

2023

2024

Mobility
$
19,832 $
25,087
26 %
Delivery

12,204
13,750
13 %
Freight

5,245
5,141...

  [2] chunk 255 rerank_score=-2.369 (RRF=0.0143)
      Financial Statements and Supplementary Data,” of this Annual Report
on Form 10-K.
Other Income (Expense), Net


Year Ended December 31,
% Change
(In m...

  [3] chunk 249 rerank_score=-3.539 (RRF=0.0301)
      ily includes the results of our share of income or loss from our
quity
method investments. For additional information, see “Note 4
Equity Method Inves...

  [4] chunk 280 rerank_score=-3.978 (RRF=0.0320)
      iabilities,
revenue and expenses, and related disclosures in our audited consolidated financial statements. We have based our estimates on
histor
cal ...

  [5] chunk 236 rerank_score=-4.160 (RRF=0.0164)
      1,273
19 %
Gross Bookings 

## Step 7: End-to-End `query_librarian()` Demo

The `query_librarian(question)` function runs the full pipeline:
Dense + BM25 → RRF → Rerank → Generate answer.

It returns the answer plus source chunks and pipeline statistics.

In [18]:
# Close the manually-opened client – query_librarian manages its own
client.close()
print("✓ Manual Weaviate client closed (query_librarian handles its own)")

✓ Manual Weaviate client closed (query_librarian handles its own)


In [19]:
result = query_librarian(
    "What was the total revenue reported in the annual report?",
    config_path=str(config_path),
    generator_mode="openai",
    verbose=True,
)

print("\n" + "=" * 70)
print(f"Answer ({result['generator_mode']}):")
print("=" * 70)
print(result['answer'])
print("\nPipeline stats:")
for k, v in result['stats'].items():
    print(f"  {k}: {v}")
print("\nTop sources:")
for i, src in enumerate(result['sources'], 1):
    print(f"  [{i}] chunk {src['chunk_id']} "
          f"(pp. {src['page_start']}-{src['page_end']}) "
          f"rerank={src['scores'].get('rerank', 'N/A')}")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6124


📚 Index already has 476 objects – skipping rebuild
🔍 Dense: 20 results | BM25: 20 results
🔀 RRF fusion: 34 unique candidates
🏆 Reranked to top-5
💬 Answer generated via 'openai'

Answer (openai):
The total revenue reported in the annual report for the year ended December 31, 2024, was $43,978 million. For the year ended December 31, 2023, the total revenue was $37,281 million.

Pipeline stats:
  dense_top_n: 20
  bm25_top_n: 20
  fused_n: 34
  reranked_k: 5
  retrieval_ms: 21.8
  generation_ms: 2362.5
  total_ms: 8610.6

Top sources:
  [1] chunk 259 (pp. 0-0) rerank=-1.1304060220718384
  [2] chunk 255 (pp. 0-0) rerank=-2.369044780731201
  [3] chunk 249 (pp. 0-0) rerank=-3.5391502380371094
  [4] chunk 280 (pp. 0-0) rerank=-3.9780781269073486
  [5] chunk 236 (pp. 0-0) rerank=-4.159510612487793


## Step 8: Entity-Heavy Queries (Why Hybrid Search Matters)

Financial documents contain specific entities that pure semantic search
often misses. These queries test the hybrid retrieval advantage.

In [20]:
entity_queries = [
    "What information is disclosed in the Form 10-K filing?",
    "What are the key financial metrics for fiscal year 2024?",
    "What are the main business segments and their revenue contributions?",
    "What risk factors are highlighted in the annual report?",
    "What does the report say about stock-based compensation?",
]

print("📊 Testing entity-heavy queries with query_librarian()\n")
for q in entity_queries:
    print(f"Q: {q}")
    r = query_librarian(q, config_path=str(config_path), generator_mode="openai", verbose=False)
    print(f"A: {r['answer'][:300]}{'...' if len(r['answer']) > 300 else ''}")
    print(f"   [{r['stats']['total_ms']:.0f}ms | {r['stats']['reranked_k']} sources]\n")

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6273


📊 Testing entity-heavy queries with query_librarian()

Q: What information is disclosed in the Form 10-K filing?


INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6321


A: The Form 10-K filing contains forward-looking statements regarding Uber Technologies, Inc.'s future results of operations, financial condition, business strategy, and management objectives. It includes a performance graph comparing the cumulative total return to shareholders on Uber's common stock r...
   [6197ms | 5 sources]

Q: What are the key financial metrics for fiscal year 2024?


INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6390


A: For the fiscal year 2024, the key financial metrics include a general and administrative expense increase of $957 million, or 36%, primarily due to a $753 million rise in legal-related accruals and expenses. Depreciation and amortization expenses decreased by $112 million, or 14%, totaling $711 mill...
   [7505ms | 5 sources]

Q: What are the main business segments and their revenue contributions?


INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6441


A: The company's main business segments are Mobility, Delivery, and Freight. For the year ended December 31, 2024, Mobility generated revenue of $25,087 million, Delivery contributed $13,750 million, and Freight accounted for $5,141 million. The total revenue for the company in 2024 was $43,978 million...
   [5457ms | 5 sources]

Q: What risk factors are highlighted in the annual report?


INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 6489


A: The annual report highlights several risk factors that could adversely affect the company's business, financial condition, and operating results. Key risks include the potential classification of Drivers as employees rather than independent contractors, which could impact operational costs. The mobi...
   [6164ms | 5 sources]

Q: What does the report say about stock-based compensation?
A: The report states that stock-based compensation expense is allocated based on the cost center to which the award holder belongs. For the years ended December 31, 2022, 2023, and 2024, total stock-based compensation expenses were $1.793 billion, $1.935 billion, and $1.796 billion, respectively. The c...
   [6663ms | 5 sources]



## Summary

**What we built:**

| Component | Implementation |
|-----------|---------------|
| **Vector Database** | Weaviate (embedded mode) |
| **Dense Retrieval** | SentenceTransformer `all-MiniLM-L6-v2` → Weaviate `near_vector` |
| **Sparse Retrieval** | Weaviate built-in BM25 |
| **Fusion** | Reciprocal Rank Fusion (RRF, k=60) |
| **Reranking** | CrossEncoder `ms-marco-MiniLM-L-6-v2` |
| **Generation** | OpenAI gpt-4o / Intern fine-tuned / Base model |
| **Entrypoint** | `query_librarian(question)` → answer + sources + stats |

**Key Insights:**
- Hybrid search (dense + BM25) catches entities that pure vector search misses
- RRF elegantly combines rankings without needing score calibration
- Cross-encoder reranking provides high-precision final ordering
- Fine-tuning the generator on domain data can improve answer quality

**Artifacts:**
- `src/rag/` – reusable RAG modules
- `.weaviate/` – embedded Weaviate persistence
- `config/config.yaml` – all parameters under `rag.*`